In [2]:
import wikipedia
import re
from pymystem3 import Mystem
from tqdm import tqdm
from multiprocessing import Pool
import pickle
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

wikipedia.set_lang('ru')

In [28]:
page_names = ['Трезубец_Посейдона', 'Флаг Барбадоса', 'Биологическое_оружие',
'Блэкберн,_Люк_Прайор', 'Рим', 'Аппиева_дорога_при_закате_солнца']

wikipedia_pages = [wikipedia.page(page_name) for page_name in page_names]

In [10]:
queries = [
    'Пейзаж с выжженной равниной увековечил образ вечного города.',
    'Будущего губернатора Кентукки обвиняли в подготовке применения биологического оружия.',
    'Трезубец на жёлто-синем фоне — это флаг Барбадоса.'
]

In [29]:
docs = []
original_sentences = []
m = Mystem()
for page in wikipedia_pages:
    s = re.sub('\s', ' ', page.content) # remove newlines and tabs
    s = re.sub('\=\= Примечания[\w\s\=]*', '', s) # remove all text after 'Примечания'
    s = re.sub(r'\=\=.*\=\=', ' ', s) # remove headers
    s = re.sub(r'[^\w \.]', '', s.lower()) # remove non-letter symbols
    s = [doc.strip() for doc in s.split('.')]
    original_sentences += s
    with Pool(4) as p:
        lemmitized = list(tqdm(p.imap(m.lemmatize, s), total=len(s)))
    # lemmitized = [m.lemmatize(doc) for doc in s]
    docs += [[lemma for lemma in doc if lemma.isalpha() or lemma.isdigit()] for doc in lemmitized]

100%|██████████| 31/31 [00:32<00:00,  1.04s/it]


In [ ]:
queries_processed = []
for query in tqdm(queries):
    s = m.lemmatize(query.lower())
    s = [lemma for lemma in s if lemma.isalpha() or lemma.isdigit()]
    queries_processed.append(s)

In [31]:
SAVE_DATA = False
if SAVE_DATA:
    with open('lemmatized2.pkl', 'wb') as output_file:
        pickle.dump((docs, original_sentences, queries_processed), output_file)

In [4]:
with open('lemmatized2.pkl', 'rb') as output_file:
    docs, original_sentences, queries_processed = pickle.load(output_file)

In [5]:
vectorizer = TfidfVectorizer(preprocessor=' '.join)
X = vectorizer.fit_transform(docs)

queries_tfidf = vectorizer.transform(queries_processed)
query_results = cosine_similarity(X, queries_tfidf)
query_results.argmax(axis=0)

array([101,  55,  44], dtype=int64)

In [42]:
for i, query in enumerate(queries):
    print(f"Query:\n{query}\n")
    print("Results:\n  Weight  | Sentence")
    for ind in query_results[:,i].argsort()[:-4:-1]:
        print(f"({query_results[ind,i]:.6f})  {original_sentences[ind]}")
        # print(f"With weight {query_results[ind,i]:.6f}")
    print("------\n")

Query:
Пейзаж с выжженной равниной увековечил образ вечного города.

Results:
  Weight  | Sentence
(0.357645)  рим стали часто называть вечным лат
(0.228971)  также рим называют городом на семи холмах
(0.197148)  roma ˈroːma  столица и крупнейший город италии
------

Query:
Будущего губернатора Кентукки обвиняли в подготовке применения биологического оружия.

Results:
  Weight  | Sentence
(0.566765)  биологическое оружие
(0.274301)  в 1879 году он был избран губернатором кентукки от демократической партии
(0.225297)  разновидностями биологического оружия являются энтомологическое оружие которое использует насекомых для атаки противника и генетическое оружие предназначенное для избирательного поражения населения по расовому этническому половому или иному генетически обусловленному признаку
------

Query:
Трезубец на жёлто-синем фоне — это флаг Барбадоса.

Results:
  Weight  | Sentence
(0.523687)  флаг барбадоса  официальный символ государства барбадос
(0.134013)  посейдон со своей женой